In [1]:
import polars as pl
import duckdb

In [2]:
with duckdb.connect("../soil_plasmid.db") as con:
    df = con.execute(
        'SELECT pOTU, Length, oids, "Ecosystem Subtype Custom" FROM ptu_derep'
    ).pl()
    df_env = con.execute(
        'SELECT taxon_oid, "Ecosystem Subtype Custom" FROM taxon_metadata'
    ).pl()

In [3]:
df.head()

pOTU,Length,oids,Ecosystem Subtype Custom
str,i64,str,str
"""pOTU-00000""",187,"""3300056587,330…","""Peat;Grassland…"
"""pOTU-00001""",158,"""3300056791,330…","""Peat;Peat;Peat…"
"""pOTU-00002""",105,"""3300046709,330…","""Agricultural l…"
"""pOTU-00003""",98,"""3300046559,330…","""Rhizosphere;Rh…"
"""pOTU-00004""",85,"""3300056834,330…","""Peat;Peat;Peat…"


In [4]:
df = df.with_columns(
    n_unique_ecos=pl.col("Ecosystem Subtype Custom")
    .str.split(";")
    .list.unique()
    .list.len(),
    n_unique_oids=pl.col("oids").str.split(",").list.unique().list.len(),
)

df.head()

pOTU,Length,oids,Ecosystem Subtype Custom,n_unique_ecos,n_unique_oids
str,i64,str,str,u32,u32
"""pOTU-00000""",187,"""3300056587,330…","""Peat;Grassland…",7,125
"""pOTU-00001""",158,"""3300056791,330…","""Peat;Peat;Peat…",3,65
"""pOTU-00002""",105,"""3300046709,330…","""Agricultural l…",5,100
"""pOTU-00003""",98,"""3300046559,330…","""Rhizosphere;Rh…",7,95
"""pOTU-00004""",85,"""3300056834,330…","""Peat;Peat;Peat…",6,71


In [5]:
oids_per_env = df_env.group_by("Ecosystem Subtype Custom").agg(
    pl.col("taxon_oid").n_unique().alias("unique_oids")
)
oids_per_env.head()

Ecosystem Subtype Custom,unique_oids
str,u32
"""Unclassified""",3642
"""Peat""",258
"""Temperate fore…",778
"""Rhizosphere""",856
"""Agricultural l…",528


In [6]:
df.with_columns(pl.col("oids").str.split(",").list.unique()).explode("oids").select(
    pl.col("pOTU"), pl.col("oids")
).join(
    df_env.select(pl.col("taxon_oid"), pl.col("Ecosystem Subtype Custom")),
    left_on="oids",
    right_on="taxon_oid",
).group_by("Ecosystem Subtype Custom").agg(
    pl.col("oids").n_unique().alias("n_unique_oids_with_pOTUs")
).join(oids_per_env, on="Ecosystem Subtype Custom").with_columns(
    (pl.col("n_unique_oids_with_pOTUs") / pl.col("unique_oids")).alias(
        "fraction_covered"
    )
)

Ecosystem Subtype Custom,n_unique_oids_with_pOTUs,unique_oids,fraction_covered
str,u32,u32,f64
"""Unclassified""",3412,3642,0.936848
"""Peat""",258,258,1.0
"""Temperate fore…",775,778,0.996144
"""Rhizosphere""",842,856,0.983645
"""Agricultural l…",523,528,0.99053
"""Tropical fores…",151,154,0.980519
"""Grasslands""",211,216,0.976852
"""Other""",694,706,0.983003


In [7]:
df.with_columns(pl.col("oids").str.split(",").list.unique()).explode("oids").select(
    pl.col("pOTU"), pl.col("oids")
).join(
    df_env.select(pl.col("taxon_oid"), pl.col("Ecosystem Subtype Custom")),
    left_on="oids",
    right_on="taxon_oid",
)

pOTU,oids,Ecosystem Subtype Custom
str,str,str
"""pOTU-00000""","""3300020809""","""Temperate fore…"
"""pOTU-00000""","""3300050137""","""Peat"""
"""pOTU-00000""","""3300048809""","""Peat"""
"""pOTU-00000""","""3300048764""","""Peat"""
"""pOTU-00000""","""3300031718""","""Temperate fore…"
"""pOTU-00000""","""3300043661""","""Other"""
"""pOTU-00000""","""3300048815""","""Peat"""
"""pOTU-00000""","""3300050116""","""Peat"""
"""pOTU-00000""","""3300060357""","""Temperate fore…"


In [8]:
df

pOTU,Length,oids,Ecosystem Subtype Custom,n_unique_ecos,n_unique_oids
str,i64,str,str,u32,u32
"""pOTU-00000""",187,"""3300056587,330…","""Peat;Grassland…",7,125
"""pOTU-00001""",158,"""3300056791,330…","""Peat;Peat;Peat…",3,65
"""pOTU-00002""",105,"""3300046709,330…","""Agricultural l…",5,100
"""pOTU-00003""",98,"""3300046559,330…","""Rhizosphere;Rh…",7,95
"""pOTU-00004""",85,"""3300056834,330…","""Peat;Peat;Peat…",6,71
"""pOTU-00005""",85,"""3300056791,330…","""Peat;Peat;Peat…",5,47
"""pOTU-00006""",83,"""3300056870,330…","""Peat;Peat;Peat…",3,51
"""pOTU-00007""",82,"""3300027857,330…","""Unclassified;R…",7,70
"""pOTU-00008""",77,"""3300028793,330…","""Other;Other;Ot…",4,77


In [11]:
df.with_columns(
    pl.col("Ecosystem Subtype Custom").str.split(";"), pl.col("oids").str.split(",")
).select(pl.col("pOTU"), pl.col("Ecosystem Subtype Custom"), pl.col("oids")).explode(
    ["Ecosystem Subtype Custom", "oids"]
).group_by(["pOTU", "Ecosystem Subtype Custom"]).agg(
    pl.col("oids").n_unique().alias("n_unique_oids")
).sort(by=["pOTU", "n_unique_oids"], descending=[False, True])

pOTU,Ecosystem Subtype Custom,n_unique_oids
str,str,u32
"""pOTU-00000""","""Peat""",52
"""pOTU-00000""","""Temperate fore…",50
"""pOTU-00000""","""Other""",8
"""pOTU-00000""","""Unclassified""",8
"""pOTU-00000""","""Agricultural l…",3
"""pOTU-00000""","""Rhizosphere""",2
"""pOTU-00000""","""Grasslands""",2
"""pOTU-00001""","""Peat""",62
"""pOTU-00001""","""Tropical fores…",2
